# Preparing

### Initialization

In [1]:
transcript_to_locus = {}
transcript_to_gene = {}
locus_to_transcripts = {}
gene_to_transcripts = {}
locus_to_genes = {}
gene_to_locus = {}
########
cdhit_to_loci = {}
cdhit_to_genes = {}
transcript_to_cdhit = {}
cdhit_to_transcripts = {}
gene_to_cdhits = {}
locus_to_cdhits = {}
########
kCluster_to_loci = {}
kCluster_to_genes = {}
transcript_to_kCluster = {}
kCluster_to_transcripts = {}
gene_to_kCluster = {}
locus_to_kCluster = {}
########
kCluster_to_cdhits = {}
cdhit_to_kClusters = {}

### Parsing Fasta File

In [2]:
with open("clstr95_loci_protein_coding_gencode.v28.transcripts.fa", "r") as fa:
    for line in fa:
        if line[0] != ">":
            continue

        line = line[1:-2].split("|")

        transcript = line[0]
        gene = line[1]
        locus = line[8]
        cdhit = line[9]

        transcript_to_gene[transcript] = gene
        transcript_to_locus[transcript] = locus
        gene_to_locus[gene] = locus
        transcript_to_cdhit[transcript] = cdhit

        # locus_to_genes
        if locus not in locus_to_genes:
            locus_to_genes[locus] = [gene]
        else:
            locus_to_genes[locus].append(gene)

        # locus_to_cdhits
        if locus not in locus_to_cdhits:
            locus_to_cdhits[locus] = [cdhit]
        else:
            locus_to_cdhits[locus].append(cdhit)

        # locus_to_transcripts
        if locus not in locus_to_transcripts:
            locus_to_transcripts[locus] = [transcript]
        else:
            locus_to_transcripts[locus].append(transcript)

        # gene_to_transcripts
        if gene not in gene_to_transcripts:
            gene_to_transcripts[gene] = [transcript]
        else:
            gene_to_transcripts[gene].append(transcript)

        # cdhit_to_genes
        if cdhit not in cdhit_to_genes:
            cdhit_to_genes[cdhit] = [gene]
        else:
            cdhit_to_genes[cdhit].append(gene)

        # cdhit_to_transcripts
        if cdhit not in cdhit_to_transcripts:
            cdhit_to_transcripts[cdhit] = [transcript]
        else:
            cdhit_to_transcripts[cdhit].append(transcript)

        # cdhit_to_loci
        if cdhit not in cdhit_to_loci:
            cdhit_to_loci[cdhit] = [locus]
        else:
            cdhit_to_loci[cdhit].append(locus)

        # gene_to_cdhits
        if gene not in gene_to_cdhits:
            gene_to_cdhits[gene] = [cdhit]
        else:
            gene_to_cdhits[gene].append(cdhit)

### Parsing Clusters File

In [3]:
with open("clusters_c100.0_UNORD_PC.tsv", "r") as clusters:
    next(clusters)
    for cluster in clusters:
        cluster = cluster.split()
        kCluster = cluster[0]
        transcripts = cluster[1].split(",")
        kCluster_to_loci[kCluster] = [transcript_to_locus[tr] for tr in transcripts]
        kCluster_to_genes[kCluster] = [transcript_to_gene[tr] for tr in transcripts]
        kCluster_to_transcripts[kCluster] = transcripts

        for tr in transcripts:
            gene = transcript_to_gene[tr]
            locus = transcript_to_locus[tr]
            gene_to_kCluster[gene] = kCluster
            locus_to_kCluster[locus] = kCluster
            transcript_to_kCluster[tr] = kCluster
        

### Parsing Comparison File

In [4]:
with open("cdhit95_kCluster100%_k25_comparison.tsv", "r") as kC:
    next(kC).split()
    for line in kC:
        line = line.split()
        q1 = line[1]
        q2 = line[2]
        kCluster = line[5]
        cdhits = line[6].split(",")
        kCluster_to_cdhits[kCluster] = cdhits



In [5]:
for kCluster, cdhits in kCluster_to_cdhits.iteritems():
    for cdhit in cdhits:
        if cdhit not in cdhit_to_kClusters:
            cdhit_to_kClusters[cdhit] = [kCluster]
        else:
            cdhit_to_kClusters[cdhit].append(kCluster)

# Benchmarking

## ***Q1. Splitted CDHIT Clusters from one kCluster***
<h4>
<ol>
    <li>How many kClusters divided into many CDHIT clusters?</li>
    <li>For every splitted kCLuster, How many complete loci in it?</li>
    <li>For every splitted kCLuster, How many complete gene in it?</li>
    <li>If there is an incomplete loci, if it contains one gene, how many transcripts are missing?</li>
    <li>If there is an incomplete gene, how many of its transcripts got grouped in both clusters' types? </li>
    <li>Does all transcripts in kCluster found in all CDHIT clusters combined?</li>
    <li>Are the CDHIT clusters grouping all transcripts of a single gene?</li>
    <li>If there is more than one locus forming the kCluster, how many of them is complete?</li>
    <li>If there is more than one gene forming the kCluster, how many of them is complete? and how many transcripts are missing from the incomplete to be complete?</li>
    <li></li>
    <li></li>
    <li></li>
    <li></li>
    <li></li>
    <li></li>
    <li></li>
</ol>
</h4>

In [8]:
no_of_splitted_kClusters = 0
for kCluster in kCluster_to_cdhits.keys():
    cdhits = kCluster_to_cdhits[kCluster]
    if len(cdhits) == 1:
        continue
    
    no_of_splitted_kClusters += 1
    loci_in_kCluster = kCluster_to_loci[kCluster]
    kCluster_number_of_loci = len(loci_in_kCluster)
    
    loci_in_cdhits = []
    for cdhit in cdhits:
        loci_in_cdhits += cdhit_to_loci[cdhit]
    
    cdhit_number_of_loci = len(loci_in_cdhits)
    
    all_tr_in_kCluster_one_loci = False
    
    
    # Check if kCluster transcripts belongs to just one locus
    if kCluster_number_of_loci == 1:
        # Number of transcripts in the kCluster
        no_of_trs = len(kCluster_to_transcripts[kCluster])
        # Check if all transcripts belongs to one gene
        if len(kCluster_to_genes[kCluster]) == 1:
            # Belongs to one gene
            kCluster_gene = kCluster_to_genes[kCluster]
            gene_trs = gene_to_transcripts[kCluster_gene]
            # Check if there are missing transcripts in that gene
            if len(gene_trs) == no_of_trs:
                # Yes It has got all transcripts
                print "Gene has got all its transcripts"
                
            else:
                diff = no_of_trs - len(gene_trs)
                # <diff> transcripts are missing to be a complete gene
        
    
    
    for locus in loci_in_kCluster:
        locus_tr = locus_to_transcripts[locus]
        number_of_locus_tr = len(locus_tr)
        locus_genes = locus_to_genes[locus]
        number_of_locus_genes = len(locus_genes)
        
        
            
    


    